<a href="https://colab.research.google.com/github/LatiefDataVisionary/mobile-legends-sentiment-analysis/blob/main/notebooks/Scraping_Data_Proyek.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **BAB 1: Persiapan Lingkungan & Library**

In [5]:
# Menginstal library google-play-scraper
!pip install google-play-scraper -q

In [6]:
# @title Import Library
import pandas as pd
import numpy as np
from google_play_scraper import Sort, reviews
from tqdm import tqdm  # Untuk progress bar agar terlihat profesional
import matplotlib.pyplot as plt
import seaborn as sns

# Konfigurasi tampilan pandas agar kolom terlihat jelas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

## **BAB 2: Konfigurasi Target Scraping**

In [7]:
# Konfigurasi Parameter Scraping
APP_ID = 'com.mobile.legends'  # ID Paket Mobile Legends di Play Store
LANG = 'id'                    # Bahasa Indonesia
COUNTRY = 'id'                 # Negara Indonesia
TARGET_COUNT = 40000           # Target data mentah (dilebihkan dari 10k)

print(f"Target Aplikasi: {APP_ID}")
print(f"Bahasa/Negara: {LANG}/{COUNTRY}")
print(f"Target Jumlah Data: {TARGET_COUNT}")

Target Aplikasi: com.mobile.legends
Bahasa/Negara: id/id
Target Jumlah Data: 40000


## **BAB 3: Eksekusi Scraping**

In [8]:
print(f" Sedang mengambil {TARGET_COUNT} ulasan dari Google Play Store...")
print(" Proses ini mungkin memakan waktu 1-3 menit bergantung pada koneksi internet...")

# Melakukan scraping
result, continuation_token = reviews(
    APP_ID,
    lang=LANG,
    country=COUNTRY,
    sort=Sort.NEWEST, # Mengambil data terbaru agar relevan dengan patch game terkini
    count=TARGET_COUNT,
    filter_score_with=None # Mengambil semua rating (1-5)
)

print(f" Berhasil mengambil {len(result)} data mentah.")

 Sedang mengambil 40000 ulasan dari Google Play Store...
 Proses ini mungkin memakan waktu 1-3 menit bergantung pada koneksi internet...
 Berhasil mengambil 40000 data mentah.


## **BAB 4: Pengolahan Data & Pembersihan Awal (Data Cleaning)**

In [9]:
# Konversi ke Pandas DataFrame
df = pd.DataFrame(result)

# Menampilkan informasi awal
print(f"Jumlah baris awal: {df.shape[0]}")
print(f"Jumlah kolom awal: {df.shape[1]}")

# 1. Seleksi Kolom Penting
# Kita hanya butuh konten ulasan, rating, dan waktu untuk analisis sentimen
df_clean = df[['content', 'score', 'at', 'userName']]

# 2. Menghapus Duplikat
# Terkadang scraper mengambil data yang sama dua kali
df_clean = df_clean.drop_duplicates(subset=['content', 'userName'])

# 3. Menghapus Data Kosong (jika ada)
df_clean = df_clean.dropna(subset=['content', 'score'])

# 4. Reset Index
df_clean = df_clean.reset_index(drop=True)

print("\n" + "="*50)
print(f" JUMLAH DATA BERSIH: {len(df_clean)} sampel")
print("="*50)

# Cek apakah memenuhi syarat submission (Minimal 10.000)
if len(df_clean) >= 10000:
    print("STATUS: AMAN (Memenuhi Syarat > 10.000 data)")
else:
    print("STATUS: KURANG (Coba naikkan TARGET_COUNT di Bab 2)")

# Tampilkan 5 sampel data teratas
df_clean.head()

Jumlah baris awal: 40000
Jumlah kolom awal: 11

 JUMLAH DATA BERSIH: 35616 sampel
STATUS: AMAN (Memenuhi Syarat > 10.000 data)


,content,score,at,userName
0,selalu dapet tim trio bot. sangat tidak di anjurkan buat di mainkan/di donlod,1,2026-02-02 17:23:36,Pengguna Google
1,yang solo di kasih tim ga jelas mulu sama devnya!!!!,1,2026-02-02 17:22:51,Pengguna Google
2,GAME HAMAAAAA,1,2026-02-02 17:21:44,Pengguna Google
3,GAME HAMAAAAAA,1,2026-02-02 17:20:54,Pengguna Google
4,👍,5,2026-02-02 17:20:10,Pengguna Google


## **BAB 5: Penyimpanan Data (Export)**

In [10]:
# Nama file output
filename = "mobile_legends_reviews_dataset.csv"

# Simpan ke CSV (tanpa index angka di kiri)
df_clean.to_csv(filename, index=False)